# NP's with Mesogens

In [ ]:
from __future__ import division
import hoomd
import hoomd.md

In [ ]:
#-----Some important variables

box         = 12.5;
replicates  = 10;
np_diameter = 5.0;
t_max       = 5.0;
p_0 = ;
p_1 = ;

In [ ]:
#-------Define the unit cell

hoomd.context.initialize("");
uc = hoomd.lattice.unitcell(N = 1,
                            a1 = [box, 0, 0],
                            a2 = [0, box, 0],
                            a3 = [0, 0, box],
                            dimensions = 3,
                            position = [[0,0,0]],
                            type_name = ['M'],
                            mass = [1.0],
                            moment_inertia = [[0,
                                               1/12*1.0*8**2,
                                               1/12*1.0*8**2]],
                            orientation = [[1, 0, 0, 0]]);

#-----From the lattice we obtain a snapshot and initialize.

snap = uc.get_snapshot()
snap.replicate(replicates,replicates,replicates)
system = hoomd.init.read_snapshot(snap)

#-----Adding the NP's and it's properties

system.particles.types.add('NP');

np_0 = system.particles[0];
np_1 = system.particles[222];
np_2 = system.particles[444];
np_3 = system.particles[666];

#----- Specify diameter and moment of inertia

np_0.type = 'NP';
np_1.type = 'NP';
np_2.type = 'NP';
np_3.type = 'NP';

np_0.diameter = np_diameter;
np_1.diameter = np_diameter;
np_2.diameter = np_diameter;
np_3.diameter = np_diameter;

np_0.moment_inertia = [0,0,0];
np_1.moment_inertia = [0,0,0];
np_2.moment_inertia = [0,0,0];
np_3.moment_inertia = [0,0,0];

np_0.mass = 5.0;
np_1.mass = 5.0;
np_2.mass = 5.0;
np_3.mass = 5.0;

#-----Add the particle type for the constituent particles.

system.particles.types.add('A');

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0)
lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

lj.pair_coeff.set('NP','M', epsilon = 1.0, sigma = 3.0)
lj.pair_coeff.set('NP','A', epsilon = 1.0, sigma = 3.0)

#-----Select an standard integrator

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define two groups and make their union.

nanoparticles = hoomd.group.type(name ='Nano_Particles', type='NP')
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens)

#----Integrate using NPT

temp = hoomd.variant.linear_interp(points = [(0,0.0), (1e4, p_max)])
npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = 1.0, tau = 0.5, tauP = 1.0, P = pressure);
npt.randomize_velocities(seed = 42)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True); 

In [ ]:
hoomd.run(1e5)